# Python aplicado a Big Data
## Dia 8 - 15/03/2021
### Instrutor: Leonardo Galler

### NLP Exemplo 2

##### PNL - ou Processamento de Linguagem Natural - é uma abreviação para uma ampla gama de técnicas projetadas para ajudar as máquinas a aprender com o texto. O Processamento de Linguagem Natural capacita tudo, desde chatbots a mecanismos de pesquisa, e é usado em diversas tarefas, como análise de sentimento e tradução automática.

Vamos utilizar textos de _tweets_ para verificar desastres.

In [61]:
# Importando pacotes
import numpy as np
import pandas as pd
from sklearn import feature_extraction, linear_model, model_selection, preprocessing

In [62]:
# Lendo os dados
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [63]:
# Informação sobre os dados de treino
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [15]:
train_df.describe().T

,count,mean,std,min,25%,50%,75%,max
id,7613.0,5441.934848,3137.11609,1.0,2734.0,5408.0,8146.0,10873.0
target,7613.0,0.429660,0.49506,0.0,0.0,0.0,1.0,1.0


In [64]:
# Observando colunas
train_df.keyword.value_counts()

fatalities               45
armageddon               42
deluge                   42
sinking                  41
body%20bags              41
                         ..
forest%20fire            19
epicentre                12
threat                   11
inundation               10
radiation%20emergency     9
Name: keyword, Length: 221, dtype: int64

In [17]:
train_df.location.value_counts()

USA                       104
New York                   71
United States              50
London                     45
Canada                     29
                         ... 
SWinfo@dot.state.al.us      1
Carterville                 1
Porthcawl                   1
atlanta                     1
africa                      1
Name: location, Length: 3341, dtype: int64

In [9]:
# Descrevendo os dados
test_df.describe().T

,count,mean,std,min,25%,50%,75%,max
id,3263.0,5427.152927,3146.427221,0.0,2683.0,5500.0,8176.0,10875.0


In [12]:
# OLhando para alguma coluna
test_df.location.value_counts()

New York                     38
USA                          37
Worldwide                    16
United States                15
London                       13
                             ..
Wakefield, England            1
Rochester Minnesota           1
Homewood, PA                  1
she/her Û¢ 19 Û¢ poland     1
USA,Washington,Seattle        1
Name: location, Length: 1602, dtype: int64

In [14]:
test_df.keyword.value_counts()

deluged                  23
rubble                   22
demolished               22
seismic                  21
sirens                   21
                         ..
forest%20fire             5
radiation%20emergency     5
fatalities                5
inundation                4
epicentre                 1
Name: keyword, Length: 221, dtype: int64

In [10]:
# Informação sobre os dados de teste
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        3263 non-null   int64 
 1   keyword   3237 non-null   object
 2   location  2158 non-null   object
 3   text      3263 non-null   object
dtypes: int64(1), object(3)
memory usage: 102.1+ KB


##### Visualizando os dados
Exemplo de tweet que não é sobre desastres.

In [65]:
train_df[train_df["target"] == 0]["text"].values[1]

'I love fruits'

Exemplo de tweet que é sobre desastres.

In [66]:
train_df[train_df["target"] == 1]["text"].values[1]

'Forest fire near La Ronge Sask. Canada'

#### Construindo vetores
Teoria: as palavras contidas em cada tweet são um bom indicador de se se trata de um desastre real ou não (isso não é totalmente correto, mas é um ótimo lugar para começar).

Usaremos o CountVectorizer do scikit-learn para contar as palavras em cada tweet e transformá-las em dados que nosso modelo de aprendizado de máquina pode processar.

Observação: um vetor é, neste contexto, um conjunto de números com os quais um modelo de aprendizado de máquina pode trabalhar. Veremos um em apenas um segundo.

In [67]:
# Converta uma coleção de documentos de texto em uma matriz de contagens de tokens
count_vectorizer = feature_extraction.text.CountVectorizer()

# O que vamos olhar:
print(train_df["text"][0:5])

## Contando os primeiros 5 registros
example_train_vectors = count_vectorizer.fit_transform(train_df["text"][0:5])
example_train_vectors

0    Our Deeds are the Reason of this #earthquake M...
1               Forest fire near La Ronge Sask. Canada
2    All residents asked to 'shelter in place' are ...
3    13,000 people receive #wildfires evacuation or...
4    Just got sent this photo from Ruby #Alaska as ...
Name: text, dtype: object


<5x54 sparse matrix of type '<class 'numpy.int64'>'
	with 61 stored elements in Compressed Sparse Row format>

In [38]:
## usamos .todense () aqui porque esses vetores são "esparsos" (apenas elementos diferentes de zero são 
### mantidos para economizar espaço)
print(example_train_vectors[0].todense().shape)
print(example_train_vectors[0].todense())

(1, 54)
[[0 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0
  0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1 0]]


##### Existem 54 palavras únicas (ou "tokens") nos primeiros cinco tweets. O primeiro tweet contém apenas alguns desses tokens exclusivos - todas as contagens diferentes de zero acima são os tokens que existem no primeiro tweet.

In [69]:
# Criando um vetor para todos os tweets
train_vectors = count_vectorizer.fit_transform(train_df["text"])
print(train_vectors)
## note que NÃO estamos usando .fit_transform () aqui. Usar apenas .transform () garante
## que os tokens nos vetores de treino são os únicos mapeados para os vetores de teste -
## que os vetores de treinamento e teste usam o mesmo conjunto de tokens.

test_vectors = count_vectorizer.transform(test_df["text"])

  (0, 14003)	1
  (0, 5490)	1
  (0, 2192)	1
  (0, 18669)	1
  (0, 15678)	1
  (0, 13681)	1
  (0, 18777)	1
  (0, 6379)	1
  (0, 12141)	1
  (0, 1852)	1
  (0, 7661)	1
  (0, 19774)	1
  (0, 1851)	1
  (1, 7652)	1
  (1, 7439)	1
  (1, 13122)	1
  (1, 11091)	1
  (1, 16266)	1
  (1, 16611)	1
  (1, 3843)	1
  (2, 2192)	2
  (2, 1851)	1
  (2, 15940)	1
  (2, 2312)	1
  (2, 18971)	1
  :	:
  (7611, 11518)	1
  (7611, 16895)	1
  (7611, 1670)	1
  (7611, 9967)	1
  (7611, 18814)	1
  (7611, 13361)	1
  (7611, 3047)	2
  (7611, 4589)	1
  (7611, 14783)	1
  (7611, 18088)	1
  (7611, 9841)	1
  (7612, 18669)	1
  (7612, 3698)	1
  (7612, 3797)	1
  (7612, 9304)	1
  (7612, 4517)	1
  (7612, 13195)	1
  (7612, 12698)	1
  (7612, 11187)	1
  (7612, 13391)	1
  (7612, 1434)	1
  (7612, 20606)	1
  (7612, 9170)	1
  (7612, 15606)	1
  (7612, 21262)	1


### O modelo

Achamos que as palavras contidas em cada tweet são um bom indicador se se trata de um desastre real ou não. A presença de uma palavra específica (ou conjunto de palavras) em um tweet pode vincular diretamente a se esse tweet é real ou não.

O que estamos assumindo aqui é uma conexão linear. Então, vamos construir um modelo linear e ver!

[Ridge Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeClassifier.html) Este classificador primeiro converte os valores de destino em {-1, 1} e, em seguida, trata o problema como uma tarefa de regressão (regressão de múltiplas saídas no caso multiclasse).<br>
[Ridge](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html#sklearn-linear-model-ridge) Este modelo resolve um modelo de regressão em que a função de perda é a função de mínimos quadrados lineares e a regularização é dada pela norma 12. Também conhecida como Regressão de cume ou regularização de Tikhonov. Este estimador tem suporte integrado para regressão multivariada (ou seja, quando e é uma matriz 2d de forma (n_samples, n_targets)).

In [41]:
## Nossos vetores são realmente grandes, então queremos empurrar os pesos do nosso modelo
## em direção a 0 sem descontar completamente palavras diferentes - regressão Ridge
## é uma boa maneira de fazer isso.
clf = linear_model.RidgeClassifier()

Vamos testar nosso modelo e ver como ele se sai nos dados de treinamento. Para isso, usaremos a validação cruzada - na qual treinamos uma parte dos dados conhecidos e depois os validamos com o resto. Se fizermos isso várias vezes (com partes diferentes), podemos ter uma boa ideia de como um determinado modelo ou método funciona.

Vamos utilizar a métrica F1.

In [70]:
scores = model_selection.cross_val_score(clf, train_vectors, train_df["target"], cv=3, scoring="f1")
scores

array([0.59453669, 0.56498283, 0.64117647])

#### As pontuações acima não são terríveis! Parece que nossa suposição marcará cerca de 0,65 na tabela de classificação. Enquanto isso, vamos fazer previsões sobre nosso conjunto de treinamento e construir uma apresentação.

In [71]:
clf.fit(train_vectors, train_df["target"])

RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, normalize=False, random_state=None,
                solver='auto', tol=0.001)

In [72]:
result = pd.read_csv("dados_predict.csv")
result

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0
...,...,...
3258,10861,0
3259,10865,0
3260,10868,0
3261,10874,0


In [73]:
result["target_predict"] = clf.predict(test_vectors)

In [74]:
result.head()

,id,target,target_predict
0,0,0,0
1,2,0,1
2,3,0,1
3,9,0,0
4,11,0,1


In [75]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [76]:
# avaliando previsões (predictions)
print('accuracy_score\n',accuracy_score(result['target'], result['target_predict']))
print()
print('confusion_matrix\n',confusion_matrix(result['target'], result['target_predict']))
print()
print('classification_report\n',classification_report(result['target'], result['target_predict']))

accuracy_score
 0.6279497395035244

confusion_matrix
 [[2049 1214]
 [   0    0]]

classification_report
               precision    recall  f1-score   support

           0       1.00      0.63      0.77      3263
           1       0.00      0.00      0.00         0

    accuracy                           0.63      3263
   macro avg       0.50      0.31      0.39      3263
weighted avg       1.00      0.63      0.77      3263



/home/leogaller/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
